In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0+cu118
tiktoken version: 0.9.0


#### 2.1 Embedding

#### 2.2 Tokenizing text

- 最小化的 表意 单元
- 最小 化的 表意单 元

In [3]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
	url = ("https://raw.githubusercontent.com/rasbt/"
		"LLMs-from-scratch/main/ch02/01_main-chapter-code/"
		"the-verdict.txt")
	file_path = "the-verdict.txt"
	urllib.request.urlretrieve(url, file_path)

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
	raw_text = f.read()
print(f"总字符数: {len(raw_text)}")
print(raw_text[:99])

总字符数: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
# use whitespace as the delimiter

import re

text = "Hello, world. This, is a test."
result = re.split(r"(\s)", text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [ ]:
# use , and . and whitespace as delimiters

result = re.split(r"([,.]|\s)", text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [ ]:
# remove empty strings
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [ ]:
# more powerful 
# split with multiple delimiters

text = "Hello, world. Is this-- a test?"

result = re.split(r"([,.:;?_!\"()']|--|\s)", text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [ ]:
# 将以上分词实际应用于文本
preprocessed = re.split(r"([,.:;?!_\"()']|--|\s)", raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
# 前面的.strip() 可以去掉前导的以及尾随的空格
print(preprocessed[:30])
print(len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


#### 2.3 token ID

In [13]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [ ]:
# mapping
vocab = {token:integer for integer, token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break


('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [30]:
class SimleTokenizerV1:
	def __init__(self, vocab) -> None:
		# 用vocab 建立一个映射表
		self.str_to_int = vocab
		self.int_to_str = {int:stri for stri, int in vocab.items()}

	def encode(self, text):
		# 将text进行分词并映射为整数ID
		preprocessed = re.split(r"([,.:;?!_()\"']|--|\s)", text)
		preprocessed = [item.strip() for item in preprocessed if item.strip()]
		ids = [self.str_to_int[s] for s in preprocessed]
		return ids
	
	def decode(self, ids):
		text = " ".join([self.int_to_str[i] for i in ids])
		# 去掉多余的空格
		text = re.sub(r"\s+([,.?!\"()'])", r"\1", text)
		return text

In [31]:
tokenizer = SimleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
			Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [32]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [33]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

#### 2.4 adding special tokens

- 特殊的token标记

	- [BOS] beginning of sentence 开始

	- [EOS] end of sentence 结束

	- [PAD] padding 填充

	- [UNK] unknown 表中未有

- <|endoftext|> 结束符 分隔两个文本

In [ ]:
tokenizer = SimleTokenizerV1(vocab)
text = "Hello, do you like tea. Is this-- a test?"
tokenizer.encode(text)
# Hello 未在表中


KeyError: 'Hello'

In [36]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}
len(vocab)

1132

In [37]:
for i, item in enumerate(list(vocab.items())[-5:]):
	print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [38]:
class SimleTokenizerV2(SimleTokenizerV1):
	def __init__(self, vocab) -> None:
		super().__init__(vocab)

	def encode(self, text):
		preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
		preprocessed = [item.strip() for item in preprocessed if item.strip()]
		preprocessed = [
			item if item in self.str_to_int else "<|unk|>" 
				for item in preprocessed
		]

		ids = [self.str_to_int[s] for s in preprocessed]
		return ids
	
	def decode(self, ids):
		return super().decode(ids)

In [39]:
tokenizer = SimleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join([text1, text2])

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [41]:
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


#### 2.5 BytePair encoding